In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Alopecia"
cohort = "GSE18876"

# Input paths
in_trait_dir = "../../input/GEO/Alopecia"
in_cohort_dir = "../../input/GEO/Alopecia/GSE18876"

# Output paths
out_data_file = "../../output/preprocess/Alopecia/GSE18876.csv"
out_gene_data_file = "../../output/preprocess/Alopecia/gene_data/GSE18876.csv"
out_clinical_data_file = "../../output/preprocess/Alopecia/clinical_data/GSE18876.csv"
json_path = "../../output/preprocess/Alopecia/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Transcriptional Profile of Aging in Healthy Human Skin"
!Series_summary	"Gene expression changes were assessed from the non sun-exposed skin of the lower back of 98 healthy males aged 19-86. We show that contrary to previous thought, genome wide transcriptional activity does not display an exclusively linear correlation with ageing, but rather, in human skin, undergoes a period of significant transient change between 30 and 45 years of age. The identified transient transcriptional changes suggest a period of heightened metabolic activity and cellular damage mediated primarily through the actions of TP53 (tumour protein 53) and TNF (tumour necrosis factor). We also identified a subgroup of the population characterised by increased expression of a large group of hair follicle genes that correlates strongly with a younger age of onset and increasing severity of androgenetic alopecia."
!Series_overall_design	"Skin was collected from the lower back at 

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability 
# Based on the background information, this dataset contains transcriptional profiles from skin samples
# hybridized to exon arrays, which indicates gene expression data is available
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability

# Age is available in row 0
age_row = 0

# Gender is not explicitly mentioned, but the background information states "healthy males" only,
# so all subjects are male (constant). Therefore gender data is not useful for our analysis.
gender_row = None

# For trait (Alopecia), there's no direct mention in the sample characteristics,
# but the background information mentions a "subgroup of the population characterised by... androgenetic alopecia"
# However, we don't have this information in the sample characteristics dictionary
trait_row = None 

# 2.2 Data Type Conversion
def convert_age(age_str):
    """Convert age string to numeric value."""
    try:
        # Extract the number after the colon and space
        if ':' in age_str:
            age_val = age_str.split(': ')[1].strip()
            return float(age_val)
        else:
            return None
    except:
        return None

def convert_trait(trait_str):
    """
    Convert trait string to binary value.
    This function is defined but won't be used since trait_row is None.
    """
    return None

def convert_gender(gender_str):
    """
    Convert gender string to binary value.
    This function is defined but won't be used since gender_row is None.
    """
    return None

# 3. Save Metadata
# Determine trait data availability
is_trait_available = trait_row is not None

# Initial filtering and save the information
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# Since trait_row is None, we skip the clinical feature extraction step entirely
if trait_row is not None:
    # This block won't execute in this case since trait_row is None
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the dataframe
    preview = preview_df(selected_clinical_df)
    print("Clinical data preview:")
    print(preview)
    
    # Save the clinical data
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    selected_clinical_df.to_csv(out_clinical_data_file)
    print(f"Clinical data saved to {out_clinical_data_file}")


### Step 3: Gene Data Extraction

In [4]:
# 1. First get the file paths again to access the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Use the get_genetic_data function from the library to get the gene_data from the matrix_file
gene_data = get_genetic_data(matrix_file)

# 3. Print the first 20 row IDs (gene or probe identifiers) for future observation
print("First 20 gene/probe identifiers:")
print(gene_data.index[:20])


First 20 gene/probe identifiers:
Index(['2315554', '2315633', '2315674', '2315739', '2315894', '2315918',
       '2315951', '2316218', '2316245', '2316379', '2316558', '2316605',
       '2316746', '2316905', '2316953', '2317246', '2317317', '2317434',
       '2317472', '2317512'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# Examine the gene identifiers in the given index
# The identifiers appear to be numerical, which suggests they are not human gene symbols
# Human gene symbols typically follow specific naming conventions (e.g., BRCA1, TP53)
# These look like probe IDs that would need mapping to gene symbols

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. First get the file paths using geo_get_relevant_filepaths function
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 3. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['2315100', '2315106', '2315109', '2315111', '2315113'], 'GB_LIST': ['NR_024005,NR_034090,NR_024004,AK093685', 'DQ786314', nan, nan, 'DQ786265'], 'SPOT_ID': ['chr1:11884-14409', 'chr1:14760-15198', 'chr1:19408-19712', 'chr1:25142-25532', 'chr1:27563-27813'], 'seqname': ['chr1', 'chr1', 'chr1', 'chr1', 'chr1'], 'RANGE_GB': ['NC_000001.10', 'NC_000001.10', 'NC_000001.10', 'NC_000001.10', 'NC_000001.10'], 'RANGE_STRAND': ['+', '+', '+', '+', '+'], 'RANGE_START': ['11884', '14760', '19408', '25142', '27563'], 'RANGE_STOP': ['14409', '15198', '19712', '25532', '27813'], 'total_probes': ['20', '8', '4', '4', '4'], 'gene_assignment': ['NR_024005 // DDX11L2 // DEAD/H (Asp-Glu-Ala-Asp/His) box polypeptide 11 like 2 // 2q13 // 84771 /// NR_034090 // DDX11L9 // DEAD/H (Asp-Glu-Ala-Asp/His) box polypeptide 11 like 9 // 15q26.3 // 100288486 /// NR_024004 // DDX11L2 // DEAD/H (Asp-Glu-Ala-Asp/His) box polypeptide 11 like 2 // 2q13 // 84771 /// AK093685 // DDX11L2 // D

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify the columns for gene identifiers and gene symbols in the gene annotation data
# From the preview, 'ID' column contains the same numeric identifiers as in gene expression data
# and 'gene_assignment' contains the gene symbol information

# 2. Get gene mapping dataframe by extracting these columns
gene_mapping = get_gene_mapping(gene_annotation, prob_col='ID', gene_col='gene_assignment')

# Check the first few rows of the mapping
print("First few rows of gene mapping:")
print(gene_mapping.head())

# 3. Apply gene mapping to convert probe-level measurements to gene-level expression
gene_data = apply_gene_mapping(gene_data, gene_mapping)

# Check the first few gene symbols in the processed data
print("\nFirst 20 gene symbols after mapping:")
print(gene_data.index[:20])

# Save gene data to CSV
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"\nGene expression data saved to {out_gene_data_file}")


First few rows of gene mapping:
        ID                                               Gene
0  2315100  NR_024005 // DDX11L2 // DEAD/H (Asp-Glu-Ala-As...
1  2315106                                                ---
2  2315109                                                ---
3  2315111                                                ---
4  2315113                                                ---



First 20 gene symbols after mapping:
Index(['A-', 'A-2', 'A-52', 'A-E', 'A-I', 'A-II', 'A-IV', 'A-V', 'A0', 'A1',
       'A1-', 'A10', 'A11', 'A12', 'A13', 'A14', 'A16', 'A1BG', 'A1BG-AS',
       'A1CF'],
      dtype='object', name='Gene')



Gene expression data saved to ../../output/preprocess/Alopecia/gene_data/GSE18876.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the gene expression data
print("Normalizing gene symbols...")
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Gene data shape after normalization: {normalized_gene_data.shape}")

# Save the normalized gene data to a CSV file
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# 2. Since trait_row is None (no trait data available), we'll create an empty clinical dataframe
print("No trait data available for clinical feature extraction")
selected_clinical_df = pd.DataFrame()

# Save empty clinical data to a CSV file for consistency
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
selected_clinical_df.to_csv(out_clinical_data_file)
print(f"Empty clinical data saved to {out_clinical_data_file}")

# Create a linked dataframe with just gene data (no clinical features)
print("Creating gene data representation without clinical features...")
linked_data = normalized_gene_data.T  # Transpose to get samples as rows
print(f"Linked data shape: {linked_data.shape}")

# 3-6. Since trait data is unavailable, we'll use is_final=False in validation
# We'll skip handling missing values and bias checking since they require trait data

# Update the note to reflect the actual dataset
note = "Dataset contains gene expression data from skin samples of healthy males of different ages, as described in the study 'Transcriptional Profile of Aging in Healthy Human Skin'. The study mentions a subgroup with androgenetic alopecia, but this information is not available in the clinical annotations."

# Perform validation with is_final=False since we can't evaluate bias without trait data
is_usable = validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

print(f"Dataset usability: {is_usable}")
print("Dataset is not usable for trait-gene association studies due to missing trait data.")

Normalizing gene symbols...
Gene data shape after normalization: (18418, 98)


Normalized gene data saved to ../../output/preprocess/Alopecia/gene_data/GSE18876.csv
No trait data available for clinical feature extraction
Empty clinical data saved to ../../output/preprocess/Alopecia/clinical_data/GSE18876.csv
Creating gene data representation without clinical features...
Linked data shape: (98, 18418)
Dataset usability: False
Dataset is not usable for trait-gene association studies due to missing trait data.
